In [ ]:
#Group project

In [1]:
# import packages
library(tidyverse)
library(readxl)
library(repr)
library(tidymodels)                                    # Install data.table
library("data.table")  
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [37]:
train <- read_excel("data/Training_data.xls") %>% mutate(Class = as_factor(UNS))

In [38]:
train

STG,SCG,STR,LPR,PEG,UNS,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
0.00,0.00,0.00,0.00,0.00,very_low,very_low
0.08,0.08,0.10,0.24,0.90,High,High
0.06,0.06,0.05,0.25,0.33,Low,Low
⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.54,0.82,0.71,0.29,0.77,High,High
0.50,0.75,0.81,0.61,0.26,Middle,Middle
0.66,0.90,0.76,0.87,0.74,High,High


In [39]:
# scale data
set.seed(9999) 

data_recipe<- recipe(UNS ~ STG + SCG + STR + LPR + PEG, data = train) %>%
                       step_scale(all_predictors()) %>%
                       step_center(all_predictors())

data_scaled<- data_recipe %>% prep() %>% bake(train)
data_scaled

STG,SCG,STR,LPR,PEG,UNS
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
-1.765089,-1.678008,-1.902921,-1.7440515,-1.796705,very_low
-1.384627,-1.300582,-1.496317,-0.7767309,1.729790,High
-1.479743,-1.394939,-1.699619,-0.7364259,-0.503657,Low
⋮,⋮,⋮,⋮,⋮,⋮
0.8030235,2.190604,0.983965,-0.5752058,1.220407,High
0.6127930,1.860357,1.390569,0.7145550,-0.777940,Middle
1.3737150,2.568030,1.187267,1.7624856,1.102857,High


In [48]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) %>%
       set_engine("kknn") %>%
       set_mode("classification")
data_recipe<- recipe(Class ~ STG + SCG + STR + LPR + PEG, data = train) %>%
                       step_scale(all_predictors()) %>%
                       step_center(all_predictors())
model_fit <- workflow() %>%
                   add_recipe(data_recipe) %>%
                   add_model(knn_spec)

In [49]:
test <- read_excel("data/test_data.xlsx") %>% mutate(Class = as_factor(UNS))
test

STG,SCG,STR,LPR,PEG,UNS,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
0.00,0.10,0.50,0.26,0.05,Very Low,Very Low
0.05,0.05,0.55,0.60,0.14,Low,Low
0.08,0.18,0.63,0.60,0.85,High,High
⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.56,0.60,0.77,0.13,0.32,Low,Low
0.66,0.68,0.81,0.57,0.57,Middle,Middle
0.68,0.64,0.79,0.97,0.24,Middle,Middle


In [50]:
knn_fit_all <- model_fit %>%
             fit(data = train)
test_predictions <- predict(knn_fit_all, test)
test_predictions

.pred_class
<fct>
very_low
Low
High
⋮
Middle
Middle
Middle


In [51]:
# get prediction on test
predicted <- test %>% 
  bind_cols(predict(knn_fit_all, test)) %>% 
  bind_cols(predict(knn_fit_all, test, type = "prob"))

# quick check
predicted %>% 
  select(Class, matches(".pred"))

# test_pred <- predict(knn_fit_all, new_data = test) %>%
#   bind_cols(predict(knn_fit_all, test, type = "prob")) %>%
#   bind_cols(test %>% select(UNS))

Class,.pred_class,.pred_very_low,.pred_High,.pred_Low,.pred_Middle
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Very Low,very_low,0.8,0,0.2,0.0
Low,Low,0.0,0,0.8,0.2
High,High,0.0,1,0.0,0.0
⋮,⋮,⋮,⋮,⋮,⋮
Low,Middle,0,0.0,0.2,0.8
Middle,Middle,0,0.4,0.0,0.6
Middle,Middle,0,0.2,0.2,0.6


In [52]:
predict_accuracy <- predicted %>% accuracy(truth = Class, estimate=.pred_class)

ERROR: Error: Problem with `summarise()` input `.estimate`.
[31m✖[39m `truth` and `estimate` levels must be equivalent.
`truth`: Very Low, Low, High, Middle
`estimate`: very_low, High, Low, Middle

[34mℹ[39m Input `.estimate` is `metric_fn(truth = Class, estimate = .pred_class, na_rm = na_rm)`.
